In [22]:
from pytrends.request import TrendReq
import pandas as pd
pytrends = TrendReq(hl='en-US', tz=360)

# Choose your terms
keywords = ["growth", "ESG", "AI", "climate"]

# Build payload (monthly, over long time span)
pytrends.build_payload(kw_list=keywords, timeframe='2004-01-01 2024-12-31')

# Retrieve interest over time
trends = pytrends.interest_over_time()

# Drop non-data column
if 'isPartial' in trends.columns:
    trends = trends.drop(columns=['isPartial'])

# Optional: resample to monthly mean if needed
trends_monthly = trends.resample('M').mean()


# Preview
print(trends_monthly.tail())

            growth  ESG     AI  climate
date                                   
2024-08-31    10.0  1.0   82.0      4.0
2024-09-30    10.0  1.0   85.0      5.0
2024-10-31    10.0  1.0   93.0      5.0
2024-11-30    10.0  1.0   96.0      5.0
2024-12-31    10.0  1.0  100.0      5.0


/var/folders/47/ztkg05vx2hbdgfnfn0bh1y3w0000gn/T/ipykernel_89311/1081108024.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  trends_monthly = trends.resample('M').mean()


In [23]:
returns_df = pd.read_csv('Data/long_short_returns_clean.csv', index_col='date')#
returns = returns_df['long_short_return']
returns.index = pd.to_datetime(returns.index)
returns.index = returns.index.to_period('M').to_timestamp('M')

df = pd.concat([trends_monthly, returns], axis=1, join='inner')

import statsmodels.api as sm

X = df[['growth', 'AI', 'ESG', 'climate']]
X_sm = sm.add_constant(X)
y = df['long_short_return']

model = sm.OLS(y, X)
res = model.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:      long_short_return   R-squared (uncentered):                   0.028
Model:                            OLS   Adj. R-squared (uncentered):              0.011
Method:                 Least Squares   F-statistic:                              1.643
Date:                Fri, 27 Jun 2025   Prob (F-statistic):                       0.181
Time:                        09:35:10   Log-Likelihood:                          455.02
No. Observations:                 174   AIC:                                     -904.0
Df Residuals:                     171   BIC:                                     -894.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

/Users/justusschenk/Bachelorarbeit/.venv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
